## **Rede Neural - Regressão** <br> COC361 - Inteligência Computacional (2021.2)
### Alunos: <br> Henrique Chaves (DRE 119025571) <br> Pedro Boechat (DRE 119065050)
<hr>

### • Importação das bibliotecas

In [ ]:
# Bibliotecas padrão
from os import (
    listdir,
    makedirs
)
import pickle
from typing import Tuple

# Bibliotecas do Jupyter
from IPython.display import display

# Bibliotecas para manipulação dos dados
import kaggle
import numpy as np
import pandas as pd

# SKLearn
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import MinMaxScaler

# Tensorflow/Keras
from tensorflow.keras.layers import (
    Dense,
    Dropout,
)
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping

from scikeras.wrappers import KerasRegressor
from tensorflow.config import list_physical_devices
from keras.engine.sequential import Sequential as TypeSequential

# Bibliotecas para plot
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

# Carregamento das variáveis de ambiente
from dotenv import load_dotenv
load_dotenv()

# Número de GPUs disponíveis para o Tensorflow/Keras
print("Número de GPUs disponíveis: ", len(list_physical_devices('GPU')))

### • Download do dataset ([Link](https://www.kaggle.com/contactprad/bike-share-daily-data?select=bike_sharing_daily.csv))

In [ ]:
# Cria pasta de destino, caso não exista
makedirs("./data/regression", exist_ok=True)

# Se a pasta de destino estiver vazia, baixa os dados
if len(listdir("./data/regression/")) == 0:
    kaggle.api.dataset_download_file(
        "contactprad/bike-share-daily-data",
        "bike_sharing_daily.csv",
        "./data/regression/"
    )

### • Carregamento do dataset

In [ ]:
# Carregamento dos dados
df = pd.read_csv("./data/regression/bike_sharing_daily.csv")

### • Estudo do dataset
```
- dteday : date
- season : season (1:springer, 2:summer, 3:fall, 4:winter)
- yr : year (0: 2011, 1:2012)
- mnth : month ( 1 to 12)
- hr : hour (0 to 23)
- holiday : weather day is holiday or not (extracted from http://dchr.dc.gov/page/holiday-schedule)
- weekday : day of the week
- workingday : if day is neither weekend nor holiday is 1, otherwise is 0.
- weathersit : 
    - 1: Clear, Few clouds, Partly cloudy, Partly cloudy
    - 2: Mist + Cloudy, Mist + Broken clouds, Mist + Few clouds, Mist
    - 3: Light Snow, Light Rain + Thunderstorm + Scattered clouds, Light Rain + Scattered clouds
    - 4: Heavy Rain + Ice Pallets + Thunderstorm + Mist, Snow + Fog
- temp : Normalized temperature in Celsius. The values are divided to 41 (max)
- atemp: Normalized feeling temperature in Celsius. The values are divided to 50 (max)
- hum: Normalized humidity. The values are divided to 100 (max)
- windspeed: Normalized wind speed. The values are divided to 67 (max)
- casual: count of casual users
- registered: count of registered users
- cnt: count of total rental bikes including both casual and registered
```

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
display(df.head())
display(df.tail())

### • Análise Exploratória dos Dados

### • Limpeza dos dados

#### 1. Remover coluna `instant`

In [ ]:
# Remove coluna `instant` se for igual ao índice do dataset
if np.all(df.index == df["instant"] - 1):
    df = df.drop("instant", axis=1)

print("df shape:", df.shape)
df.sample(5)

#### 2. Converter coluna `season` para variáveis dummies

In [ ]:
dummies_season = pd.get_dummies(df["season"], drop_first=True)
dummies_season = dummies_season.rename(
    columns={
        2: "is_summer",
        3: "is_fall",
        4: "is_winter"
    }
)

print("dummies_season shape:", dummies_season.shape)
dummies_season.sample(5)

In [ ]:
df = df.drop("season", axis=1)
df = pd.concat([df, dummies_season], axis=1)
print("df shape:", df.shape)
df.sample(5)

#### 3. Converter colunas `mnth` e `weekday` para variáveis cíclicas usando `sin` e `cos`

In [ ]:
df["mnth_cos"] = np.cos(df["mnth"]*np.pi/6)
df["mnth_sin"] = np.cos(df["mnth"]*np.pi/6)
df["weekday_cos"] = np.cos((df["weekday"]+1)*2*np.pi/7)
df["weekday_sin"] = np.cos((df["weekday"]+1)*2*np.pi/7)

df = df.drop(["mnth", "weekday"], axis=1)
print("df shape:", df.shape)
df.sample(5)

#### 4. Desnormalizar colunas `temp`, `atemp`, `hum` e `windspeed`

In [ ]:
df["temp"] *= 41
df["atemp"] *= 50
df["hum"] *= 100
df["windspeed"] *= 67

print("df shape:", df.shape)
df.sample(5)

#### 5. Remover colunas `casual` e `registered` pois a soma delas é igual a `cnt` (variável alvo)

In [ ]:
if np.all(df["casual"] + df["registered"] == df["cnt"]):
    df = df.drop(["casual", "registered"], axis=1)

print("df shape:", df.shape)
df.sample(5)

### • Salvar dataset limpo

In [ ]:
df.to_csv("./data/regression/df_clean.csv", index=False)

### • Definindo `features` e  `targets`

In [ ]:
features = ["yr", "holiday", "workingday", "weathersit",
            "temp", "atemp", "hum", "windspeed",
            "is_summer", "is_fall", "is_winter",
            "mnth_cos", "mnth_sin", "weekday_cos", "weekday_sin"]

targets = ["cnt"]

In [ ]:
df_X = df[features]

print("df_X shape:", df_X.shape)
df_X.sample(5)

In [ ]:
df_y = df[targets]

print("df_y shape:", df_y.shape)
df_y.sample(5)

### • Normalização dos dados

In [ ]:
# Definição do scaler
scaler = MinMaxScaler

# Instância do scaler para X e Y
X_scaler = scaler()
y_scaler = scaler()

In [ ]:
X = X_scaler.fit_transform(df_X)
y = y_scaler.fit_transform(df_y)


print("X shape:", X.shape)
print("y shape:", y.shape)

### • Definição de callbacks da rede

In [ ]:
def create_model(
    n_hidden_layers: int,
    n_neurons: int,
    dropout_rate: float,
    dropout_last_layer: bool,
    learning_rate: float = 0.001,
    input_shape: Tuple[int, ] = (X.shape[1], )
) -> TypeSequential:
    """Função que retorna o modelo compilado a partir dos parâmetros.
    Args:
        n_layers (int): Número de camadas da rede. 2 por padrão.
        n_neurons (int): Número de neurônios da rede. 32 por padrão.
        dropout_rate (float): Taxa de dropout. 0.2 por padrão.
        dropout_last_layer (bool): Se terá dropout na última camada.
        False por padrão.
        learning_rate (float): Learning rate do modelo. 0.001 por padrão.
        input_shape (List[int]): Forma da entrada. [99] por padrão.
    """
    # Criação do modelo sequencial
    model = Sequential()

    # Número de variáveis de entrada
    model.add(
            Dense(
                n_neurons,
                activation='relu',
                input_shape=input_shape
            )
        )

    for i in range(n_hidden_layers):
        # Camada de adensamento com ativação RELU
        model.add(
            Dense(
                n_neurons,
                activation='relu'
            )
        )

        # Camada de dropout
        if dropout_rate > 0.0:
            if (i == n_hidden_layers - 1) and (not dropout_last_layer):
                continue
            model.add(
                Dropout(
                    dropout_rate
                )
            )

    # Camada de adensamento com ativação LINEAR
    model.add(Dense(1, activation='linear'))

    # Otimizador Adam
    optimizer = Adam(learning_rate=learning_rate)

    # Compilação do modelo
    model.compile(
        optimizer=optimizer,
        loss='mse',
        metrics=['mae']
    )

    return model

In [ ]:
# Reduz a learning rate caso o modelo esteja estagnado
lr_reduce = ReduceLROnPlateau(
    min_delta=1e-5,
    patience=5,
    verbose=1
)

# Lista contendo os checkpoints definidos
callbacks = [
    lr_reduce
]

### • Definição das camadas da rede

In [ ]:
# Criação do regressor com wrapper do SKLearn
regressor = KerasRegressor(
    model=create_model,
    n_hidden_layers=1,
    n_neurons=32,
    dropout_rate=0.0,
    dropout_last_layer=False,
    learning_rate=0.0001,
    batch_size=32,
    epochs=100,
)

In [ ]:
# Parâmetros para o Grid Search
param_grid = {
    "n_hidden_layers": [1, 2, 3],
    "n_neurons": [32, 64, 128],
    "dropout_rate": [0.0, 0.1, 0.2],
    "dropout_last_layer": [False, True],
    "learning_rate": np.logspace(-4, -2, 3),
}

In [ ]:
# Instância do Grid Search
grid_search = GridSearchCV(
    estimator=regressor,
    param_grid=param_grid,
    scoring='neg_mean_squared_error',
    cv=10,
    n_jobs=1
)

### • Treino da rede

In [ ]:
# Treino do modelo
grid_result = grid_search.fit(
    X, y,
    callbacks=callbacks,
    verbose=1
)

In [ ]:
with open("./data/regression/grid_result.pkl", "wb") as f:
    pickle.dump(grid_result, f)

In [ ]:
# history = grid_result.best_estimator_.model.model.history.history

### • Avaliação da rede

In [ ]:
# Definição dos subplots
fig, ax = plt.subplots(figsize=(15, 16), nrows=2)

# Gráfico do MAE do modelo por época
ax[0].plot(history['mse'])
ax[0].plot(history['val_mae'])
ax[0].set_title('MAE do modelo por época', fontsize=18)
ax[0].set_ylabel('MAE', fontsize=14)
ax[0].set_xlabel('Época', fontsize=14)
ax[0].legend(['Treino', 'Validação'], loc='upper left', fontsize=16)

# Gráfico da loss do modelo por época
ax[1].plot(history['loss'])
ax[1].plot(history['val_loss'])
ax[1].set_title('Loss (MSE) do modelo por época', fontsize=18)
ax[1].set_ylabel('Loss (MSE)', fontsize=14)
ax[1].set_xlabel('Época', fontsize=14)
ax[1].legend(['Treino', 'Validação'], loc='upper left', fontsize=16)

# Ajuste do layout do plot
plt.tight_layout()